<a href="https://colab.research.google.com/github/nsfogg/cnn-pooling-analyzer/blob/main/adl_project_code_fogg_sweasey.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## CNN Analyzer

In [ ]:
# Import torch, torch datasets, and other necessary packages
import torch
import torchvision.datasets as datasets
import torchvision.transforms as transform
import os
import sys
import numpy as py
import matplotlib.pyplot as plt

In [ ]:
# Determine what device is running through CoLab servers.
# All testing was performed using cuda under Google server's T4 GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
device

In [ ]:
# Create tensor transformation tool
toTensor = transform.ToTensor()

# Set the dataset to use, this example uses MNIST
data = datasets.MNIST

# Since FER2013 is not a dataset in torch, using it requires different steps
if data == datasets.FER2013:
  # Necessary for FER2013 dataset as dataset is not local to pytorch
  # Needs to be accessed from google drive
  from google.colab import drive

  # Connect to local google drive to view dataset csv saved there
  drive.mount('/content/drive')

  # Since using FER2013, root connects to google drive
  root = "/content/drive/MyDrive"

  # Set up train and test sets for cnn to refer to, transforming data to tensors
  train = data(root=root, split="train", transform=toTensor)
  test = data(root=root, split="test", transform=toTensor)
else:
  # Set up train and test sets for cnn to refer to, download necessary data
  train = data(root='./data', train=True, download=True, transform=toTensor)
  test = data(root='./data', train=False, download=True, transform=toTensor)

# Create labels for the cnn to classify, depending on dataset

# labels = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'] # CIFAR10
labels = list(range(0,10)) # MNIST
# labels = [1, 2, 3, 4, 5, 6] #FER2013

labels

In [ ]:
# Create iterator of train dataset to create trainable batches
train_iter = iter(train)

In [ ]:
# Collect an image and label from the train dataset
image, label = next(train_iter)

# This shows a sample image. This is what the machine is processing
image.shape
np_img = image.numpy()
print(np_img.shape)
print(labels[label])
# The size of the image depends on the how the dataset stores them
plt.imshow(np_img.reshape((28, 28, 1)))
# plt.imshow(np_img.reshape((32, 32, 3)))
# plt.imshow(np_img.transpose((1, 2, 0)))

In [ ]:
# Split training dataset into training and validation
train, validate = torch.utils.data.random_split(train, [50000, 10000])
# Display the samples in train, validate, and test sets
len(train), len(validate), len(test)

In [ ]:
# Create dataloaders for train and validate sets. Allows the data to be iterated over
import torch.utils.data.dataloader as dl

# Specifies how many images are processed per iteration
batch_size = 8

trainloader = dl.DataLoader(train, batch_size = batch_size,
                              shuffle=True, num_workers=2)
valloader = dl.DataLoader(validate, batch_size = batch_size,
                          shuffle=False, num_workers=2)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

# Build the nerual network
class NeuralNet(nn.Module):
    def __init__(self):
        super().__init__()

        # Some layers must be slightly altered depending on input size
        self.conv1 = nn.Conv2d(1, 256, 3)
        # self.conv1 = nn.Conv2d(1, 256, 2)
        self.pool1 = nn.AdaptiveAvgPool2d(2) # Changed depending on which trial is being run

        # self.conv2 = nn.Conv2d(256, 512, 3)
        self.conv2 = nn.Conv2d(256, 512, 2)
        self.pool2 = nn.AdaptiveAvgPool2d(2)

        self.conv3 = nn.Conv2d(512, 1024, 2)
        self.pool3 = nn.AdaptiveAvgPool2d(2)

        self.flatten = nn.Flatten()

        # This depends on input size. Adjusted based on error message
        self.fc1 = nn.Linear(4096, 1024)
        # self.fc1 = nn.Linear(25600, 1024)

        self.drop1 = nn.Dropout(p=0.3)

        self.fc2 = nn.Linear(1024, 1024)
        self.drop2 = nn.Dropout(p=0.3)

        self.out = nn.Linear(1024, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool1(x)

        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool2(x)

        x = self.conv3(x)
        x = F.relu(x)
        x = self.pool3(x)

        x = self.flatten(x)

        x = self.fc1(x)
        x = F.relu(x)
        x = self.drop1(x)

        x = self.fc2(x)
        x = F.relu(x)
        x = self.drop2(x)

        x = self.out(x)

        return x


In [ ]:
net = NeuralNet()
net.to(device)

In [ ]:
# Define our loss function, optimization function, and learning rate
import torch.optim as op


criterion = nn.CrossEntropyLoss()
optimizer = op.Adam(net.parameters(), lr=0.0001)

In [ ]:
# Defines how one epoch is trained
def train_epoch():
    net.train(True)

    # Keep track of loss and accuracy throughout the epoch
    running_loss = 0.0
    running_accuracy = 0.0

    # Training loop
    for batch_index, data in enumerate(trainloader):
        # Load data to appropriate device (T4 GPU in this case)
        inputs, labels = data[0].to(device), data[1].to(device)
        # Set gradient to zero
        optimizer.zero_grad()

        # Runs the neural network, generates outputs, and calculates accuracy
        outputs = net(inputs)
        correct = torch.sum(labels == torch.argmax(outputs, dim=1)).item()
        running_accuracy += correct / batch_size

        # Calculates loss and runs backpropogation
        loss = criterion(outputs, labels)
        running_loss += loss.item()
        loss.backward()
        optimizer.step()

        # Prints every 500 batches
        if batch_index % 500 == 499:
            avg_loss_across_batches = running_loss / 500
            avg_acc_across_batches = (running_accuracy / 500) * 100
            print('Batch {0}, Loss: {1:.3f}, Accuracy: {2:.1f}%'.format(batch_index+1, avg_loss_across_batches, avg_acc_across_batches))

            running_loss = 0.0
            running_accuracy = 0.0

    print('-----------------------------------------------------------------------')




In [ ]:
# Defines how one epoch is validated/tested
def validate_epoch():
    net.train(False)
    running_loss = 0.0
    running_accuracy = 0.0

    # Validation loop
    for i, data in enumerate(valloader):
        # Load data to appropriate device (T4 GPU in this case)
        inputs, labels = data[0].to(device), data[1].to(device)

        # Does not run backwards propogation; instead just calculates loss and accuracy
        with torch.no_grad():
            outputs = net(inputs)
            correct = torch.sum(labels == torch.argmax(outputs, dim=1)).item()
            running_accuracy += correct / batch_size
            loss = criterion(outputs, labels)
            running_loss += loss.item()

    avg_loss_across_batches = running_loss / len(valloader)
    avg_acc_across_batches = (running_accuracy / len(valloader)) * 100

    print('Val Loss: {0:.3f}, Val Accuracy: {1:.1f}%;'.format(avg_loss_across_batches, avg_acc_across_batches))
    print('-----------------------------------------------------------------------')

In [ ]:
# How many generations to train for. Higher number will train for longer

num_epochs = 10 # change this if you want

# Train and validate for each epoch
for epoch_index in range(num_epochs):
    print(f'Epoch: {epoch_index + 1}\n')
    train_epoch()
    validate_epoch()

print('Finished! :)')
